# Dataframe Guide

We first create our dataframe out of arrays as seen below:

In [2]:
import sys
import pandas as pd
import numpy as np

In [3]:
densities = [5000, 5000, 4000, 3000, 5000, 5000, 3000]
longlat_ind = [16290, 16290, 16380, 16475, 48870, 48800, 48870]

ids = [0,0,0,0,1,1,1]
period =[30000,30000,30000,30000,50000,50000,50000]
type = ['s','c','c','c','s','c','c']
loc = ['t1','t1','t1','t1','t2','t2','t2']
pseud_type = ['a','b','b','b','a','b','b']
contributions = [0.25, 0.25, 0.25, 0.25, 0.33, 0.33, 0.33]

data = pd.DataFrame({'location': loc, 'density':densities, 'cluster_id':ids,'period':period, 'latlon_ind': longlat_ind, 'type':type, 'pseudo_type':pseud_type, 'contribution': contributions})
data

,cluster_id,contribution,density,latlon_ind,location,period,pseudo_type,type
0,0,0.25,5000,16290,t1,30000,a,s
1,0,0.25,5000,16290,t1,30000,b,c
2,0,0.25,4000,16380,t1,30000,b,c
3,0,0.25,3000,16475,t1,30000,b,c
4,1,0.33,5000,48870,t2,50000,a,s
5,1,0.33,5000,48800,t2,50000,b,c
6,1,0.33,3000,48870,t2,50000,b,c


## Filtering and Statistics

We can easily filter the rows of our dataframes and extract statistics from its columns.

### Filtering
For example, we can get a dataframe whose columns only have a value "s"

In [4]:
sample_dataframe = data[data.type == 's']
sample_dataframe

,cluster_id,contribution,density,latlon_ind,location,period,pseudo_type,type
0,0,0.25,5000,16290,t1,30000,a,s
4,1,0.33,5000,48870,t2,50000,a,s


### Statistics

We can also get statistics such as mean from the dataframe.

For example, we want to get the mean of all densities in our original dataframe.

In [5]:
data['density'].mean()

4285.7142857142853

We can also get the mean of a filtered dataframe. For example, the mean of all dataframes with type 'c':

In [6]:
data[data.type == 'c']['density'].mean()

4000.0

Outside of just taking the average, we can also take sums or count the number of rows for a dataframe.

In [7]:
data[data.cluster_id==0]['contribution'].sum()

1.0

In [8]:
data[data.type == 's']['density'].count()

2

### Groupby

Another way of filtering and getting statistics is through the groupby function. This is useful if you want the a statistic for each type. 

For example, we want the mean density for each cluster. 

In [9]:
data.groupby('cluster_id')['density'].mean()

cluster_id
0    4250.000000
1    4333.333333
Name: density, dtype: float64

We can also group by two columns, such that rows are aggregated by unique pairs of these two columns.

For example, we can group by type and period, then get the mean for all the rest of the columns for each pair. 

In [13]:
data.groupby(['type', 'period']).mean()

cluster_id  contribution  density    latlon_ind
type period                                                 
c    30000          0.0          0.25   4000.0  16381.666667
     50000          1.0          0.33   4000.0  48835.000000
s    30000          0.0          0.25   5000.0  16290.000000
     50000          1.0          0.33   5000.0  48870.000000

Note that we could not refer to the columns we grouped in the typical way we do for other columns. For example:

In [14]:
new_data = data.groupby(['type', 'period']).mean()
new_data["density"] # this is okay

type  period
c     30000     4000.0
      50000     4000.0
s     30000     5000.0
      50000     5000.0
Name: density, dtype: float64

In [15]:
new_data["type"] # this does not work

KeyError: 'type'

To be able to refer to the columns we used as keys in the groupby command, we have to reset the index. For instance:

In [16]:
new_data = data.groupby(['type', 'period']).mean().reset_index()
new_data['type']

0    c
1    c
2    s
3    s
Name: type, dtype: object

## Adding Columns (based on other columns)
With dataframes, we can easily make a new column based on the values of other columns.

A simple example would be to make a new column with half the density:

In [12]:
data['half_density'] = data.density/2
data

,cluster_id,contribution,density,latlon_ind,location,period,pseudo_type,type,half_density
0,0,0.25,5000,16290,t1,30000,a,s,2500.0
1,0,0.25,5000,16290,t1,30000,b,c,2500.0
2,0,0.25,4000,16380,t1,30000,b,c,2000.0
3,0,0.25,3000,16475,t1,30000,b,c,1500.0
4,1,0.33,5000,48870,t2,50000,a,s,2500.0
5,1,0.33,5000,48800,t2,50000,b,c,2500.0
6,1,0.33,3000,48870,t2,50000,b,c,1500.0


This allows us to create columns to assign bins. 

For example, we can have bins with size 2000

In [35]:
bin_size = 2000
data['bin_index'] = (data.density)/bin_size
data['bin_index'] = data.bin_index.astype(int)
data['bin'] = data.bin_index*bin_size

sorted(data['bin'].unique())

[2000, 4000]

In [17]:
data

,cluster_id,contribution,density,latlon_ind,location,period,pseudo_type,type
0,0,0.25,5000,16290,t1,30000,a,s
1,0,0.25,5000,16290,t1,30000,b,c
2,0,0.25,4000,16380,t1,30000,b,c
3,0,0.25,3000,16475,t1,30000,b,c
4,1,0.33,5000,48870,t2,50000,a,s
5,1,0.33,5000,48800,t2,50000,b,c
6,1,0.33,3000,48870,t2,50000,b,c
